In [3]:
# ! pip install keras-rl2
# ! pip install chess
# ! pip install python-chess

In [4]:
import numpy as np
import gym

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten,\
     Input, Conv2D, BatchNormalization, MaxPool2D, Reshape, Dropout
from tensorflow.keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

# import gym_chess

import chess
# import sys
# sys.path.insert(0, '../')
# sys.path.insert(0, '../alpha_beta')
# from MyChessBoard import MyChessBoard

2021-12-10 21:12:44.201208: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-10 21:12:44.201237: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11868930409909203330
]


2021-12-10 21:12:45.704301: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-10 21:12:45.705063: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-10 21:12:45.705078: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-10 21:12:45.705098: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LoG): /proc/driver/nvidia/version does not exist


In [6]:
STATE_SHAPE = (65, )
NB_ACTIONS = 4096

In [7]:
class ChessEnv:
    '''
    state - obser: ndarray - (65,): [:65] is flatten from int_board; [65] is color of bot; 1 is white and -1 is black
    step: int. step_range = (0, 4096) , is encoded from square A to square B (64 x 64 val)
    reward: int
    '''

    mapped = {
            'P': 10,     # White Pawn
            'p': -10,    # Black Pawn
            'N': 20,     # White Knight
            'n': -20,    # Black Knight
            'B': 30,     # White Bishop
            'b': -30,    # Black Bishop
            'R': 40,     # White Rook
            'r': -40,    # Black Rook
            'Q': 50,     # White Queen
            'q': -50,    # Black Queen
            'K': 900,     # White King
            'k': -900     # Black King
    }
    # state_shape = (8, 8)
    # nb_actions = 4096
    model = None
    
    def __init__(self, model: Sequential, neg_r_each_step = -1) -> None:
        self.env = chess.Board()
        self.state = self.reset()
        # [-1] = 1 -> white, -1 -> black
        self.bot_color = self.env.turn * 2 - 1
        self.neg_r_each_step = neg_r_each_step
        self.model = model

    def is_draw(self):
        if self.env.is_stalemate():
            print("statlemate")
            return True
        if self.env.is_fivefold_repetition():
            print("fivefold repetition")
            return True
        if self.env.is_seventyfive_moves():
            print("75 moves")
            return True
        if self.env.is_insufficient_material():
            print("Insufficient Material")
            return True
        return False

    def is_checkmate(self):
        # If There is checkmate then it will be TRUE else FALSE.It will be a boolean value.
        return self.env.is_checkmate()

    def convert_board_to_int(self):
        epd_string = self.env.epd()
        list_int = np.empty((0, ))
        for i in epd_string:
            if i == " ":
                list_int = list_int.reshape((8, 8))
                return list_int
            elif i != "/":
                if i in self.mapped:
                    list_int = np.append(list_int, self.mapped[i])
                else:
                    for counter in range(0, int(i)):
                        list_int = np.append(list_int, 0)
        list_int = list_int.reshape((8, 8))
        return list_int

    def get_state(self) -> np.ndarray:
        return np.append(self.convert_board_to_int().reshape(64,), self.env.turn * 2 - 1)

    def legal_moves(self):
        return list(self.env.legal_moves)

    def encodeMove(self, move_uci:str):
        if len(move_uci) != 4:
            raise ValueError()
        a, b = chess.parse_square(move_uci[:2]), chess.parse_square(move_uci[2:])
        return a * 64 + b

    def decodeMove(self, move_int:int):
        a, b = move_int//64, move_int%64
        # a, b = chess.square_name(a), chess.square_name(b)

        move = self.env.find_move(from_square= a,to_square= b)
        return move

    def render(self):
        print(self.env.unicode())

    def reset(self):
        # random state
        redo = True
        num_sample_steps = 0
        while redo:
            redo = False
            self.env = chess.Board()
            num_sample_steps = np.random.randint(0, 50)
            for i in range (num_sample_steps):
                lg_move = self.legal_moves()
                if len(lg_move) != 0:
                    move = np.random.choice(self.legal_moves())
                    self.env.push(move)
                else:
                    redo = True
                    break
        return self.get_state()

    def step(self, action: int):
        reward = 0
        done = True

        try:
            # move in legal move
            move = self.decodeMove(action)

            # neg reward each step
            reward = self.neg_r_each_step

            # location to_square
            to_r, to_c = move.to_square//8, move.to_square%8
            reward -= self.state[(7 - to_r)*8 + to_c ] * self.bot_color

            # action
            self.env.push(move)
            self.state = self.get_state()

            # check end game
            if self.is_checkmate():
                reward += self.mapped['K']
                done = True
            elif self.is_draw():
                reward += 300
                done = True

            # opponent's turn   
            else:
                done = False
                Q_val = self.model.predict(self.state.reshape((1, 1) + STATE_SHAPE)).reshape(-1, )
                idx_sorted = np.argsort(Q_val)

                for act in idx_sorted:
                    try:
                        move = self.decodeMove(act)

                        # location to_square
                        to_r, to_c = move.to_square//8, move.to_square%8
                        reward -= self.state[(7 - to_r)*8 + to_c ] * self.bot_color

                        # action
                        self.env.push(move)
                        self.state = self.get_state()

                        # check end game
                        if self.is_checkmate():
                            reward -= self.mapped['K']
                            done = True
                        elif self.is_draw():
                            reward += 300
                            done = True
                        
                        break
                    except:
                        continue

        except:
            # wrong move
            reward = -5000
            done = True
            print('wrong_move')

        return self.state, reward, done, {}


In [8]:
# model
model = Sequential()
model.add(Input((1, ) + STATE_SHAPE))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(NB_ACTIONS))
model.add(Activation('linear'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 65)                0         
_________________________________________________________________
dense (Dense)                (None, 128)               8448      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 128)               512       
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
__________________________________________________

In [9]:
env = ChessEnv(model, neg_r_each_step=-1)

In [11]:
# Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
# even the metrics!
memory = SequentialMemory(limit=500000, window_length=1)
policy = EpsGreedyQPolicy(0.3)
dqn = DQNAgent(model=model, nb_actions=NB_ACTIONS, memory=memory, nb_steps_warmup=10,
               target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
his = dqn.fit(env, nb_steps=500000, visualize=False, verbose=2)


/home/kienanh/.local/lib/python3.9/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Training for 50000 steps ...


/home/kienanh/.local/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


wrong_move
     1/50000: episode: 1, duration: 0.531s, episode steps:   1, steps per second:   2, episode reward: -5000.000, mean reward: -5000.000 [-5000.000, -5000.000], mean action: 3337.000 [3337.000, 3337.000],  loss: --, mae: --, mean_q: --
wrong_move
     2/50000: episode: 2, duration: 0.010s, episode steps:   1, steps per second:  98, episode reward: -5000.000, mean reward: -5000.000 [-5000.000, -5000.000], mean action: 714.000 [714.000, 714.000],  loss: --, mae: --, mean_q: --
wrong_move
     3/50000: episode: 3, duration: 0.014s, episode steps:   1, steps per second:  69, episode reward: -5000.000, mean reward: -5000.000 [-5000.000, -5000.000], mean action: 821.000 [821.000, 821.000],  loss: --, mae: --, mean_q: --
wrong_move
     4/50000: episode: 4, duration: 0.005s, episode steps:   1, steps per second: 194, episode reward: -5000.000, mean reward: -5000.000 [-5000.000, -5000.000], mean action: 2151.000 [2151.000, 2151.000],  loss: --, mae: --, mean_q: --
wrong_move
     5/

/home/kienanh/.local/lib/python3.9/site-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
/home/kienanh/.local/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


    11/50000: episode: 11, duration: 2.039s, episode steps:   1, steps per second:   0, episode reward: -5000.000, mean reward: -5000.000 [-5000.000, -5000.000], mean action: 192.000 [192.000, 192.000],  loss: --, mae: --, mean_q: --
wrong_move
    12/50000: episode: 12, duration: 0.039s, episode steps:   1, steps per second:  26, episode reward: -5000.000, mean reward: -5000.000 [-5000.000, -5000.000], mean action: 367.000 [367.000, 367.000],  loss: 12884251.000000, mae: 145.099060, mean_q: 246.914200
wrong_move
    13/50000: episode: 13, duration: 0.042s, episode steps:   1, steps per second:  24, episode reward: -5000.000, mean reward: -5000.000 [-5000.000, -5000.000], mean action: 404.000 [404.000, 404.000],  loss: 12843257.000000, mae: 147.297974, mean_q: 261.431335
wrong_move
    14/50000: episode: 14, duration: 0.034s, episode steps:   1, steps per second:  30, episode reward: -5000.000, mean reward: -5000.000 [-5000.000, -5000.000], mean action: 1835.000 [1835.000, 1835.000],  

/home/kienanh/.local/lib/python3.9/site-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
/home/kienanh/.local/lib/python3.9/site-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
/home/kienanh/.local/lib/python3.9/site-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
/home/kienanh/.local/lib/python3.9

    16/50000: episode: 16, duration: 0.054s, episode steps:   1, steps per second:  19, episode reward: -5000.000, mean reward: -5000.000 [-5000.000, -5000.000], mean action: 192.000 [192.000, 192.000],  loss: 12726982.000000, mae: 145.589767, mean_q: 251.247650
wrong_move
    17/50000: episode: 17, duration: 0.072s, episode steps:   1, steps per second:  14, episode reward: -5000.000, mean reward: -5000.000 [-5000.000, -5000.000], mean action: 2130.000 [2130.000, 2130.000],  loss: 12819499.000000, mae: 147.727295, mean_q: 260.152527
wrong_move
    18/50000: episode: 18, duration: 0.075s, episode steps:   1, steps per second:  13, episode reward: -5000.000, mean reward: -5000.000 [-5000.000, -5000.000], mean action: 2130.000 [2130.000, 2130.000],  loss: 12692618.000000, mae: 149.395477, mean_q: 265.783813
wrong_move
    19/50000: episode: 19, duration: 0.043s, episode steps:   1, steps per second:  23, episode reward: -5000.000, mean reward: -5000.000 [-5000.000, -5000.000], mean actio

/home/kienanh/.local/lib/python3.9/site-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


    20/50000: episode: 20, duration: 0.036s, episode steps:   1, steps per second:  27, episode reward: -5000.000, mean reward: -5000.000 [-5000.000, -5000.000], mean action: 192.000 [192.000, 192.000],  loss: 12799388.000000, mae: 149.282928, mean_q: 254.349869
wrong_move
    21/50000: episode: 21, duration: 0.050s, episode steps:   1, steps per second:  20, episode reward: -5000.000, mean reward: -5000.000 [-5000.000, -5000.000], mean action: 2844.000 [2844.000, 2844.000],  loss: 12727290.000000, mae: 145.682129, mean_q: 235.561615
wrong_move
    22/50000: episode: 22, duration: 0.058s, episode steps:   1, steps per second:  17, episode reward: -5000.000, mean reward: -5000.000 [-5000.000, -5000.000], mean action: 2130.000 [2130.000, 2130.000],  loss: 12787268.000000, mae: 147.561737, mean_q: 250.121490
wrong_move
    23/50000: episode: 23, duration: 0.057s, episode steps:   1, steps per second:  18, episode reward: -5000.000, mean reward: -5000.000 [-5000.000, -5000.000], mean actio

In [ ]:
# # save dqn
# dqn.save_weights('dqn_{}_weights.h5f'.format('chess'), overwrite=True)

# # save model
# model.save('chess_model.h5')

In [ ]:
# from tensorflow import keras
# model = keras.models.load_model('chess_model.h5')

In [184]:
env.state

array([[ -40.,  -20.,  -30.,  -50., -900.,  -30.,  -20.,  -40.],
       [ -10.,  -10.,  -10.,  -10.,  -10.,  -10.,  -10.,  -10.],
       [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
       [  10.,   10.,   10.,   10.,   10.,   10.,   10.,   10.],
       [  40.,   20.,   30.,   50.,  900.,   30.,   20.,   40.]])

In [189]:
pred = model.predict(env.state.reshape((1, 1) + STATE_SHAPE))
idx_sort = np.argsort(pred)